In [1]:
import json_lines as jl
import nltk
import xmltodict
from ispras import texterra as isp
import pymorphy2 as pm2
import gensim
import time as tm

isp_api = isp.API('ba74236a7212a71054ae1408b30b1bdef771d35b')
#nltk.download()

tokenizer = nltk.TweetTokenizer()
morph = pm2.MorphAnalyzer()  

C:\Users\Maxim\Anaconda3\lib\site-packages\gensim-2.2.0-py3.5-win-amd64.egg\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Лемматизация с помощью Texterra

Читаю комментарии из файла по очереди, разбиваю каждый на предложения, лемматизирую с помощью Texterra, составляя при этом словарь FullVocab. В словаре хранятся связи <u>"нормальная форма"</u> - <u>"id нормальной формы"</u> - <u>"часть речи"</u> - <u>"встретившиеся варианты слова"</u>. В lemmatized.txt записываются построчно лемматизированные предложения, но вместо слов в нём <u>"id нормальной формы"</u>, чтобы для word2vec слова стать (<i>глаол<>)

In [ ]:
class FullVocab:
    
    def __init__(self, **value):
        self.id_counter = 0
        self.vocab = []
        if value['from_file'] == 'yes':
            f = open('vocab.xml', 'r')
            root = etree.XML(f.read())
            for item in root:
                self.add_item(item[3].text.split(' '), item[1].text, item[2].text)
            
        
    def add_item(self, words, normal_form, tags_normal):
        if type(words) is not list:
            words = [words]
            
        for item in self.vocab:
            if item.get('normal_form') == normal_form and item.get('tags') == tags_normal:
                item['word_forms'].extend(words)
                item['word_forms'] = sorted(set(item['word_forms']))
                return item['id']
                
        new_item = {'id': self.id_counter, 'normal_form': normal_form, 'tags': tags_normal, 'word_forms': words}
        self.vocab.append(new_item)
        self.id_counter = self.id_counter + 1
        return self.id_counter - 1
            
    def show_vocab(self):
        print(self.vocab)
        
    def build_xml(self):
        f = open('vocab.xml', 'w')
        root = etree.Element('root')
        for item in self.vocab:
            child = etree.SubElement(root, 'w' + str(item['id']))
            etree.SubElement(child, 'id').text = str(item['id'])
            etree.SubElement(child, 'normal_form').text = item['normal_form']
            etree.SubElement(child, 'tags').text = item['tags']
            etree.SubElement(child, 'word_forms'). text = ' '.join(item['word_forms'])
            
        f.write(etree.tostring(root, encoding='unicode', method='xml', pretty_print=True))


In [51]:
lemmatized_txt = open('lemmatized.txt', 'w', encoding='utf8')

parse_normal_dict = list()

with open('learn_scrapy\pda.jl', 'rb') as f:
    for comment in jl.reader(f):
        sents = nltk.sent_tokenize(comment)
        for sent in sents:
            toks = tknzr.tokenize(sent)
            toks = [t.lower() for t in toks if t.isalnum()]
            sent = ' '.join(toks)
            
            lemma_raw = isp_api.lemmatizationAnnotate(text = sent)
            lemmatized = ''
            if lemma_raw.get('annotations').get('lemma') is not None:
                
                for l in lemma_raw.get('annotations').get('lemma'):
                    lemmatized = lemmatized + l.get('value') + ' '

'на и понимать разуметься '

# Some testing

In [12]:
comments = ''
with open('learn_scrapy\pda.jl', 'rb') as f:
    for item in jl.reader(f):
        comments = comments + item['comment']       #создаём один string на все комменты
                                                    #скорее всего, будет логичнее для N-граммов каждое предложение
                                                    #запоминать в один string и хранить как list()

In [13]:
#тут токенизируем комментарии
#есть подозрение, что я создаю слишком много сущностей 

tknzr = nltk.TweetTokenizer()
print('Kek1')
tokens = tknzr.tokenize(comments)
print('Kek2')
text = nltk.Text(tokens)
print('Kek3')
sents = nltk.sent_tokenize(comments)                         #строить n-граммную модель, возможно, буду 
print('Kek4')                                                             #анализируя предложения по-отдельности
words = sorted([w.lower() for w in tokens if w.isalpha()])   #впоследствии буду отсекать наименее наиболее частотные слова

Kek1
Kek2
Kek3
Kek4


In [20]:
def lexical_diversity(text):
    return len(set(text)) / len(text)

print(lexical_diversity(text))

0.10600020178435018


In [21]:
sent_tokens = list()                            #здесь храним предложения
for i in range(len(sents)):
    sent_tokens.append(tknzr.tokenize(sents[i]))    

In [22]:
#сох нормальные формы слов
morph = pm2.MorphAnalyzer()     
vocab_normalized = sorted(set([morph.parse(w)[0].normalized for w in words]), key = lambda w: w.normal_form)   

In [ ]:
def progress(full, curr, checkpoint):
    if curr / full * 100 > checkpoint:
        print(curr / full * 100, end = '')
        print('% complited')
        return True
    
    else:
        return False
    
i = 0
h = 0.1
checkpoint = h
time = tm.clock()
for sent in sents:
    if progress(len(sents), i, checkpoint):
        time_per_checkpoint = tm.clock() - time
        print('Time per checkpoint:', end = ' ')
        print(time_per_checkpoint)
        print('Time per sentence:', end = ' ')
        time_per_sent = time_per_checkpoint / (len(sents) * h)
        print(time_per_sent)
        print('Estimated time:', end = ' ')
        print(time_per_checkpoint * (100 - checkpoint))
        time = tm.clock()
        checkpoint = checkpoint + h
        
    toks = tknzr.tokenize(sent)
    toks = [t.lower() for t in toks if t.isalnum()]
    sentence = ''
    for t in toks:
        sentence = sentence + t + ' '
    
    lemma_raw = isp_api.lemmatizationAnnotate(text = sentence)
    lemmatized = ''
    if lemma_raw.get('annotations').get('lemma') is not None:
        for l in lemma_raw.get('annotations').get('lemma'):
            lemmatized = lemmatized + l.get('value') + ' '
        
        sents[i] = lemmatized
    
    else:
        sents[i] = ''
        
    i = i + 1
        
    
sents[0]

In [2]:
with open('lemmatized.txt', 'w', encoding='utf8') as f:
    for s in sents[0:5000]:
        f.write(s + '\n')

NameError: name 'sents' is not defined

In [11]:
model = gensim.models.Word2Vec(gensim.models.word2vec.LineSentence('lemmatized.txt'), min_count=10)


In [93]:
model.most_similar(positive = ['айфон'])

[('а', 0.9998720288276672),
 ('они', 0.9998717904090881),
 ('где', 0.9998696446418762),
 ('этот', 0.9998689293861389),
 ('мочь', 0.9998672604560852),
 ('когда', 0.999864935874939),
 ('это', 0.9998648166656494),
 ('там', 0.9998632669448853),
 ('с', 0.9998587965965271),
 ('ничто', 0.9998581409454346)]

In [5]:
i = 5

message = 'сейчас я говорю о том человеке который спит отдыхая на работе'
message_tokens = tokenizer.tokenize(message)
parse_tokens = [morph.parse(t) for t in message_tokens]
#print(parse_tokens[i])
isp_POS = isp_api.posTaggingAnnotate(text = message)
isp_lemma = isp_api.lemmatizationAnnotate(text = message)

#print(isp_POS['annotations']['pos-token'][i])

#print(isp_lemma.get('annotations').get('lemma')[i].get('value'))

#print(' '.join([l.get('value') for l in isp_lemma.get('annotations').get('lemma')]))

for j in range(len(message_tokens)):
    isp_lemma_token = isp_lemma.get('annotations').get('lemma')[j].get('value')
    morph_parse_token = [w.normalized for w in morph.parse(isp_lemma_token) if w.normal_form == w.word]
    print(isp_lemma_token)
    print(isp_POS['annotations']['pos-token'][j])
    print(morph_parse_token)
    print("\n")

сейчас
{'value': {'characters': [], 'tag': 'ADV', 'type': 'syn-tag-rus'}, 'start': 0, 'text': 'сейчас', 'end': 6, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='сейчас', tag=OpencorporaTag('ADVB'), normal_form='сейчас', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'сейчас', 3, 0),))]


я
{'value': {'characters': [{'tag': 'Masculine', 'type': 'gender'}, {'tag': 'DEICTIC', 'type': 'pronoun'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'Nominative', 'type': 'case'}, {'tag': 'Animated', 'type': 'animacy'}], 'tag': 'S', 'type': 'syn-tag-rus'}, 'start': 7, 'text': 'я', 'end': 8, 'annotated-text': 'сейчас я говорю о том человеке который спит отдыхая на работе'}
[Parse(word='я', tag=OpencorporaTag('NPRO,1per sing,nomn'), normal_form='я', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'я', 3100, 0),))]


говорить
{'value': {'characters': [{'tag': 'First', 'type': 'person'}, {'tag': 'Singular', 'type': 'number'}, {'tag': 'NotPast', 'type

In [76]:
ws = ['встать', 'вставать', 'встали', 'вставали']
for w in ws:
    print(w)
    ns = morph.parse(w)
    ts = isp_api.lemmatizationAnnotate(text = w)
    print(ts)
    print(ns)
        
    print('\n')

встать
{'text': 'встать', 'annotations': {'lemma': [{'text': 'встать', 'value': 'вставать', 'annotated-text': 'встать', 'start': 0, 'end': 6}]}}
[Parse(word='встать', tag=OpencorporaTag('INFN,perf,intr'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встать', 904, 0),))]


вставать
{'text': 'вставать', 'annotations': {'lemma': [{'text': 'вставать', 'value': 'вставать', 'annotated-text': 'вставать', 'start': 0, 'end': 8}]}}
[Parse(word='вставать', tag=OpencorporaTag('INFN,impf,intr'), normal_form='вставать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'вставать', 903, 0),))]


встали
{'text': 'встали', 'annotations': {'lemma': [{'text': 'встали', 'value': 'вставать', 'annotated-text': 'встали', 'start': 0, 'end': 6}]}}
[Parse(word='встали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='встать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'встали', 904, 4),))]


вставали
{'text': 'вставали', 'annotations': {'lemma': [{'text': 'встав